In [1]:
import requests
import json

class The_national_map_USGS():

    def __init__(self):
        self.api_endpoint = r'https://tnmaccess.nationalmap.gov/api/v1/'
        self.datasets = self.datasets_full()
    
    def datasets_full(self):
        try:
            return requests.get(f'{self.api_endpoint}datasets?').json()
        except:
            print('Failed to load metadata from The National Map API endpoint V1')
            return []
    @property
    def datatypes(self):
        return set(i['displayName'] for ds in self.datasets for i in ds['formats'])
    
    @property
    def collections(self):
        return [y['sbDatasetTag'] for x in self.datasets for y in x['tags']]

def find_tiles(collection=None, datatype=None, keyword=None, region=[]):
    TNM = The_national_map_USGS()
    
    if collection:
        assert collection in TNM.collections, f'Unknown collection, must be element of {TNM.collections}'
    if datatype:
        assert datatype in TNM.datatypes, f'Unknown datatypes, must be element of {TNM.datatypes}'
 
    def clean_list(x):
        return ','.join(' '.join(map(str,point)) for point in x)
    
    parameters = {'prodFormats': datatype,
                  'datasets': collection,
                  'q':keyword,
                  'polygon': clean_list(region)}
    
    response = requests.get(f'{TNM.api_endpoint}products?', params=parameters)
    if response.status_code//100 == 2:
        return response.json()
    return {}

U = The_national_map_USGS()
U.collections

['US Topo Current',
 'US Topo Historical',
 'National Elevation Dataset (NED) 1 arc-second',
 'Digital Elevation Model (DEM) 1 meter',
 'National Elevation Dataset (NED) 1/3 arc-second',
 'National Elevation Dataset (NED) 1/9 arc-second',
 'National Elevation Dataset (NED) Alaska 2 arc-second',
 'Alaska IFSAR 5 meter DEM',
 'National Elevation Dataset (NED) 1/3 arc-second - Contours',
 'Original Product Resolution (OPR) Digital Elevation Model (DEM)',
 'Ifsar Digital Surface Model (DSM)',
 'Ifsar Orthorectified Radar Image (ORI)',
 'Lidar Point Cloud (LPC)',
 'National Hydrography Dataset Plus High Resolution (NHDPlus HR)',
 'National Hydrography Dataset (NHD) Best Resolution',
 'National Watershed Boundary Dataset (WBD)',
 'Small-scale Datasets - Boundaries',
 'Small-scale Datasets - Contours',
 'Small-scale Datasets - Hydrography',
 'Small-scale Datasets - Transportation',
 'Topobathymetric Lidar DEM',
 'Topobathymetric Lidar Point Cloud']

In [2]:
example_url = r"https://tnmaccess.nationalmap.gov/api/v1/products?prodFormats=LAS,LAZ&datasets=Lidar%20Point%20Cloud%20(LPC)&polygon=-104.94262695312236%2041.52867510196275,-102.83325195312291%2041.52867510196275,-102.83325195312291%2040.45065268246805,-104.94262695312236%2040.45065268246805,%20-104.94262695312236%2041.52867510196275&"
example_url = requests.get(example_url).json()    

paras = {'datatype':'LAS,LAZ', 
         'collection':'Lidar Point Cloud (LPC)',
         'region': [
                     (-104.94262695312236, 41.52867510196275),
                     (-102.83325195312291, 41.52867510196275),
                     (-102.83325195312291, 40.45065268246805),
                     (-104.94262695312236, 40.45065268246805),
                     (-104.94262695312236, 41.52867510196275),
                   ]
        }

U = The_national_map_USGS()
z = find_tiles(**paras)
example_url['total'] == z['total']

True

In [3]:
paras = {'datatype':'LAS,LAZ', 
         'keyword':'LPC',
         'region': [
                     (-104.94262695312236, 41.52867510196275),
                     (-102.83325195312291, 41.52867510196275),
                     (-102.83325195312291, 40.45065268246805),
                     (-104.94262695312236, 40.45065268246805),
                     (-104.94262695312236, 41.52867510196275),
                   ]
        }

U = The_national_map_USGS()
z = find_tiles(**paras)
example_url['total'] == z['total']

True

In [4]:
z['items'][0]

{'title': 'USGS Lidar Point Cloud CO SoPlatteRiver-Lot1 2013 13TFE627477 LAS 2015',
 'moreInfo': 'Lidar (Light detection and ranging) discrete-return point cloud data are available in the American Society for Photogrammetry and Remote Sensing (ASPRS) LAS format. The LAS format is a standardized binary format for storing 3-dimensional point cloud data and point attributes along with header information and variable length records specific to the data. Millions of data points are stored as a 3-dimensional data cloud as a series of x (longitude), y (latitude) and z (elevation) points. A few older projects in this collection are in ASCII format. Please refer to http://www.asprs.org/Committee-General/LASer-LAS-File-Format-Exchange-Activities.html for additional information.',
 'sourceId': '5a801f46e4b00f54eb2a10dc',
 'sourceName': 'ScienceBase',
 'sourceOriginId': None,
 'sourceOriginName': 'gda',
 'metaUrl': 'https://www.sciencebase.gov/catalog/item/5a801f46e4b00f54eb2a10dc',
 'vendorMetaUr